# 1. Imports and file/directory setup

In [11]:
import os

from os import remove
from os.path import join, splitext
import numpy as np
import pandas as pd
from convert_eprime.utils import remove_unicode
from convert_eprime.convert import text_to_csv

data_dir = '/Volumes/Stephan_WD/NEUFEPME_data/'
org_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/'
bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_BIDS/'
eprime_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_eprime/'

os.chdir(bids_data_dir)
all_subs = next(os.walk(bids_data_dir))[1]
sub = '012'
sub_data_dir = eprime_data_dir + 'sub-' + sub
txt_files = [None] * 4
paradigms = ['motor-', 'motor-imagine-', 'emotion-', 'emotion-imagine-']
paradigm_nrs = ['-001', '-002', '-001', '-002']
csv_files = [None] * 4
event_files = [None] * 4
for i, par in enumerate(paradigms):
    txt_files[i] = join(sub_data_dir, 'neufepme-' + par + sub + paradigm_nrs[i] + '.txt')
    name, ext = splitext(txt_files[i])
    csv_files[i] = name + '.csv'
    event_files[i] = name + '_events.csv'

# 2. Read text files, convert to csv files, and load dataframes

In [12]:
all_dfs = {}
for i, fn in enumerate(txt_files):
    text_to_csv(fn, csv_files[i])
    df = pd.read_csv(csv_files[i])
    all_dfs[paradigms[i]] = df

Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-motor-012-001.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-motor-imagine-012-002.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-emotion-012-001.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-emotion-imagine-012-002.csv


# 3. Extract necessary data from dataframes, reorder into sensible blocks, write to text file

## 3.1) Motor, motor-imagine, and emotion-imagine (same structure or eprime files)

In [7]:
## Description  of data to extract:
# Rows 0:9 (including 9) - ['Rest.OnsetTime', 'Rest.OnsetToOnsetTime', 'Tap.OnsetTime']
# Row 10 - ['LastRest.OnsetTime']

In [13]:
# paradigms = ['motor-', 'motor-imagine-', 'emotion-', 'emotion-imagine-']

task = ['tapping', 'mental-tapping', '', 'mental-emotion']

for p in [0,1,3]:    
    df = all_dfs[paradigms[p]]
    # print(df)
    columns = ['type', 'onset']
    events_df = pd.DataFrame(columns=columns)

    for i in np.arange(10):
        events_df = events_df.append({'type': 'rest', 'onset': df.loc[i, 'Rest.OnsetTime']}, ignore_index=True)
        events_df = events_df.append({'type': task[p], 'onset': df.loc[i, 'Tap.OnsetTime']}, ignore_index=True)

    events_df = events_df.append({'type': 'rest', 'onset': df.loc[10, 'LastRest.OnsetTime']}, ignore_index=True)
    
    events_df.to_csv(event_files[p], index=False)
    print(task[p])
    print(events_df)


tapping
       type     onset
0      rest   69205.0
1   tapping   89218.0
2      rest  109230.0
3   tapping  129243.0
4      rest  149256.0
5   tapping  169268.0
6      rest  189281.0
7   tapping  209294.0
8      rest  229306.0
9   tapping  249319.0
10     rest  269332.0
11  tapping  289345.0
12     rest  309357.0
13  tapping  329370.0
14     rest  349383.0
15  tapping  369395.0
16     rest  389408.0
17  tapping  409421.0
18     rest  429433.0
19  tapping  449446.0
20     rest  469459.0
mental-tapping
              type     onset
0             rest   67815.0
1   mental-tapping   87827.0
2             rest  107840.0
3   mental-tapping  127853.0
4             rest  147865.0
5   mental-tapping  167878.0
6             rest  187891.0
7   mental-tapping  207903.0
8             rest  227916.0
9   mental-tapping  247929.0
10            rest  267941.0
11  mental-tapping  287954.0
12            rest  307967.0
13  mental-tapping  327979.0
14            rest  347992.0
15  mental-tapping  368005.0


## 3.2) Emotion

In [14]:
## Description  of data to extract:

# { Cue+Match(shape)+5*[ITI+Match(shape)] + Cue+Match(face)+5*[ITI+Match(face)] }*10 + Cue+Match(shape)+5*[ITI+Match(shape)]

# --0--
# 0-4 = ShapeTaskList:ShapesDisplay.OnsetToOnsetTime
# 5 = ShapesList:Shape.OnsetToOnsetTime
# 6-10 = FaceTaskList:FacesDisplay.OnsetToOnsetTime
# 11 = FacesList:Face.OnsetToOnsetTime
# --1--
# 12-16 = ShapeTaskList1:ShapesDisplay1.OnsetToOnsetTime
# 17 = ShapesList1:Shape1.OnsetToOnsetTime
# 18-22 = FaceTaskList1:FacesDisplay1.OnsetToOnsetTime
# 23 = FacesList1:Face1.OnsetToOnsetTime
# --2--

In [15]:
# (1) Extract interpretable blocks

df = all_dfs[paradigms[2]]
all_blocks = {}

for i in np.arange(11):
    loc1 = i*12
    loc2 = loc1+4
    loc3 = loc2+1
    loc4 = loc3+1
    loc5 = loc4+4
    loc6 = loc5+1
    
    all_blocks[i] = {}
    
    if i == 0:
        txt = ''
    else:
        txt = str(i)
    
    marker1a = 'ShapeTaskList' + txt
    marker1b = 'ShapesDisplay' + txt + '.OnsetToOnsetTime'
    marker2a = 'ShapesList' + txt
    marker2b = 'Shape' + txt + '.OnsetToOnsetTime'
    marker3a = 'FaceTaskList' + txt
    marker3b = 'FacesDisplay' + txt + '.OnsetToOnsetTime'
    marker4a = 'FacesList' + txt
    marker4b = 'Face' + txt + '.OnsetToOnsetTime'
    
    if i < 10:
        all_blocks[i]['shape1'] = df.loc[loc3, marker2a:marker2b]
        all_blocks[i]['shape5'] = df.loc[loc1:loc2, marker1a:marker1b]
        all_blocks[i]['face1'] = df.loc[loc6, marker4a:marker4b]
        all_blocks[i]['face5'] = df.loc[loc4:loc5, marker3a:marker3b]
    else:        
        all_blocks[i]['shape1'] = df.loc[loc3, marker2a:marker2b]
        all_blocks[i]['shape5'] = df.loc[loc1:loc2, marker1a:marker1b]

In [16]:
# (2) Test some (potentially problematic) blocks; in sub-001, these are likely still problems: ShapesCue0, FacesCue2
print(all_blocks[0]['shape1'])
print(all_blocks[2]['face1'])

ShapesList                        1
Stimulus                        NaN
ShapesList.Cycle                  1
ShapesList.Sample                 1
ShapesCue.OnsetDelay             17
ShapesCue.OnsetTime           81852
ShapesCue.DurationError           0
ShapesCue.RTTime                  0
ShapesCue.ACC                     1
ShapesCue.RT                      0
ShapesCue.RESP                  NaN
ShapesCue.CRESP                 NaN
ShapesCue.OnsetToOnsetTime     3012
Shape.OnsetDelay                 12
Shape.OnsetTime               84864
Shape.DurationError               1
Shape.RTTime                  86381
Shape.ACC                         0
Shape.RT                       1517
Shape.RESP                        r
Shape.CRESP                     NaN
Shape.OnsetToOnsetTime         2008
Name: 5, dtype: object
FacesList2                         1
FacesList2.Cycle                   1
FacesList2.Sample                  1
FacesCue2.OnsetDelay               8
FacesCue2.OnsetTime           183254


In [17]:
# (3) Add rows of data iteratively to new dataframe, and write dataframe to "events" file
columns = ['type', 'onset', 'OnsetToOnsetTime', 'response', 'RT', 'OnsetDelay']
events = pd.DataFrame(columns=columns)

def appendRowToDf(main_df, block_df, event_type, txt_col):
    main_df = main_df.append({'type':event_type,
                            'onset':block_df[txt_col + '.OnsetTime'],
                            'OnsetToOnsetTime':block_df[txt_col + '.OnsetToOnsetTime'],
                            'response':block_df[txt_col + '.RESP'],
                            'RT':block_df[txt_col + '.RT'],
                            'OnsetDelay':block_df[txt_col + '.OnsetDelay']}, ignore_index = True)
    return main_df

def appendRowToDfProblem(main_df, block_df, event_type, txt_col):
    main_df = main_df.append({'type':event_type,
                            'onset':np.nan,
                            'OnsetToOnsetTime':np.nan,
                            'response':np.nan,
                            'RT':np.nan,
                            'OnsetDelay':np.nan}, ignore_index = True)
    return main_df

for i in np.arange(11):
    loc1 = i*12
    loc2 = loc1+4
    loc3 = loc2+1
    loc4 = loc3+1
    loc5 = loc4+4
    loc6 = loc5+1
    
    if i == 0:
        txt = ''
    else:
        txt = str(i)
    
    block_df = all_blocks[i]['shape1']
    
    event_type = 'cue-shapes'
    txt_col = 'ShapesCue' + txt
    events = appendRowToDf(events, block_df, event_type, txt_col)
#     if i in [0]:
#         events = appendRowToDfProblem(events, block_df, event_type, txt_col)
#     else:
#         events = appendRowToDf(events, block_df, event_type, txt_col)
    
    event_type = 'shapes'
    txt_col = 'Shape' + txt
    events = appendRowToDf(events, block_df, event_type, txt_col)

    # all_blocks[i]['shape5']
    for j in range(loc1,loc2+1):
        block_df = all_blocks[i]['shape5'].loc[j]
        event_type = 'ITI'
        txt_col = 'ShapeRest' + txt
        events = appendRowToDf(events, block_df, event_type, txt_col)        
        event_type = 'shapes'
        txt_col = 'ShapesDisplay' + txt
        events = appendRowToDf(events, block_df, event_type, txt_col)
    
    if i < 10:
        # all_blocks[i]['face1']
        block_df = all_blocks[i]['face1']

        event_type = 'cue-faces'
        txt_col = 'FacesCue' + txt
        events = appendRowToDf(events, block_df, event_type, txt_col)
#         if i in [2]:
#             events = appendRowToDfProblem(events, block_df, event_type, txt_col)
#         else:
#             events = appendRowToDf(events, block_df, event_type, txt_col)

        event_type = 'faces'
        txt_col = 'Face' + txt
        events = appendRowToDf(events, block_df, event_type, txt_col)

        # all_blocks[i]['face5']
        for j in range(loc4,loc5+1):
            block_df = all_blocks[i]['face5'].loc[j]
            event_type = 'ITI'
            txt_col = 'FaceRest' + txt
            events = appendRowToDf(events, block_df, event_type, txt_col)        
            event_type = 'faces'
            txt_col = 'FacesDisplay' + txt
            events = appendRowToDf(events, block_df, event_type, txt_col)

print(events)
events.to_csv(event_files[2], index=False)

           type     onset  OnsetToOnsetTime response      RT  OnsetDelay
0    cue-shapes   81852.0            3012.0      NaN     0.0        17.0
1        shapes   84864.0            2008.0        r  1517.0        12.0
2           ITI   86872.0            1054.0      NaN     0.0         8.0
3        shapes   87926.0               0.0        r  1102.0        54.0
4           ITI   89934.0            1054.0      NaN     0.0         8.0
..          ...       ...               ...      ...     ...         ...
247      shapes  499206.0               0.0        g   626.0        54.0
248         ITI  501214.0            1055.0      NaN     0.0         8.0
249      shapes  502269.0               0.0        g  1235.0        55.0
250         ITI  504277.0            1054.0      NaN     0.0         8.0
251      shapes  505331.0               0.0        g   685.0        54.0

[252 rows x 6 columns]


# Extra unused code

In [ ]:
# with open(text_file, 'r', encoding="utf8", errors='ignore') as fo:
#     raw_data = fo.readlines()[:20]
#     raw_data = [l.rstrip() for l in raw_data]

# # Remove unicode characters.
# filtered_data = [remove_unicode(row) for row in raw_data]

# print('The raw text file (after removing unicode characters):')
# for l in filtered_data:
#     print(l)
# print('')